In [17]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# PNU별 평당 분양가
sql = '''
with date as (
    select '202208' as base_ym
)
select A.pnu, A.emd_cd, A.price
from (-- PNU별 가치환원법
	select pnu, substring(pnu,1,10) as emd_cd, round((min_price_1 + max_price_1)/2,0) as price
	from m2.cremao_price A -- 예측 분양가
	where base_ym = (select base_ym from date)
	) A
join m1.pnu B -- 실제 PNU 확인
	on substring(A.pnu, 1, 19) = B.pnu
join (-- 실제 건물 확인	
	select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
	from m1.bld_rgst
	where base_ym = (select base_ym from date)
) C
	on substring(A.pnu, 1, 19) = C.pnu
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','emd_cd','price']
df['price'] = df['price'].astype(int)

# 작업일 및 업데이트일, 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_price'

# Delete DATA
aws_cursor.execute(f'delete from {table} where 1=1')
aws_conn.commit()

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-16 19:04:58.090739  :  0
2022-11-16 19:04:59.986090  :  10000
2022-11-16 19:05:03.767923  :  20000
2022-11-16 19:05:06.070195  :  30000
2022-11-16 19:05:08.008183  :  40000
2022-11-16 19:05:09.828129  :  50000
2022-11-16 19:05:11.750272  :  60000
2022-11-16 19:05:13.497683  :  70000
2022-11-16 19:05:15.387709  :  80000
2022-11-16 19:05:17.192141  :  90000
2022-11-16 19:05:18.905907  :  100000
2022-11-16 19:05:20.670705  :  110000
2022-11-16 19:05:22.456200  :  120000
2022-11-16 19:05:24.467763  :  130000
2022-11-16 19:05:26.322254  :  140000
2022-11-16 19:05:28.166625  :  150000
2022-11-16 19:05:29.804380  :  160000
2022-11-16 19:05:31.756594  :  170000
2022-11-16 19:05:33.486995  :  180000
2022-11-16 19:05:35.372284  :  190000
2022-11-16 19:05:37.010901  :  200000
2022-11-16 19:05:38.851754  :  210000
2022-11-16 19:05:40.831020  :  220000
2022-11-16 19:05:42.851603  :  230000
2022-11-16 19:05:44.955721  :  240000
2022-11-16 19:05:46.808815  :  250000
2022-11-16 19:05:48.696171